<a href="https://colab.research.google.com/github/Balaji-731/AI_Internship/blob/main/Shork_Vosk_vs_Whisper_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Shork Vosk vs Whisper Script
!pip install -q vosk gTTS faster-whisper pydub soundfile sentencepiece
!apt -qq install -y ffmpeg

from google.colab import files
from gtts import gTTS
from IPython.display import Audio
import os, wave, json, subprocess
## Audio Prep
print("Upload audio (optional). Skip to auto create tts sample.")
up=files.upload()

if up:
  audio=list(up.keys())[0]
else:
  tts=gTTS("Hello! This is a test audio for comparing Vosk and Whisper.", lang="en")
  audio="sample.mp3"
  tts.save(audio)
## convert->WAV 16khz mono
wav="audio.wav"
subprocess.run(["ffmpeg", "-y","-i", audio, "-ar", "16000", "-ac", "1", wav],
               stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
Audio(wav)

## vosk
if not os.path.exists("vosk-model"):
  print("Downloading Vosk model...")
  !rm -f model.zip # Ensure a clean slate
  !wget https://alphacephei.com/vosk/models/vosk-model-en-us-0.22.zip -O model.zip
  !unzip model.zip
  !mv vosk-model-en-us-0.22 vosk-model
  !rm model.zip

from vosk import Model, KaldiRecognizer
wf=wave.open(wav, "rb")
rec=KaldiRecognizer(Model("vosk-model"), wf.getframerate())
vosk_text=""

while True:
  data=wf.readframes(4000)
  if not data: break
  if rec.AcceptWaveform(data):
    vosk_text+=json.loads(rec.Result()).get("text"," ")+" "
vosk_text+=json.loads(rec.FinalResult()).get("text"," ")

print("\n>>> VOSK:\n",vosk_text.strip())

## whisper

import torch
from faster_whisper import WhisperModel
device="cuda" if torch.cuda.is_available() else "cpu"
wmodel=WhisperModel("small", device=device)

segments,_=wmodel.transcribe(wav)
whisper_text=" ".join([s.text for s in segments]).strip()

print("\n>>> WHISPER:\n",whisper_text)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Upload audio (optional). Skip to auto create tts sample.


--2025-11-23 11:13:40--  https://alphacephei.com/vosk/models/vosk-model-en-us-0.22.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1913365522 (1.8G) [application/zip]
Saving to: ‘model.zip’

model.zip           100%[===================>]   1.78G  27.6MB/s    in 68s     

2025-11-23 11:14:48 (27.0 MB/s) - ‘model.zip’ saved [1913365522/1913365522]

Archive:  model.zip
   creating: vosk-model-en-us-0.22/
   creating: vosk-model-en-us-0.22/am/
  inflating: vosk-model-en-us-0.22/am/final.mdl  
  inflating: vosk-model-en-us-0.22/am/tree  
   creating: vosk-model-en-us-0.22/ivector/
  inflating: vosk-model-en-us-0.22/ivector/final.dubm  
  inflating: vosk-model-en-us-0.22/ivector/final.ie  
  inflating: vosk-model-en-us-0.22/ivector/final.mat  
  inflating: vosk-model-en-us-0.22/ivector/splice.conf  
  inflating: vosk-mode

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]


>>> WHISPER:
 Hello, this is a test audio for comparing Vosk and Whisper.
